In the following project I intend to do a time series analysis of volatility in Twitter stock. Specifically, I will see if the implied volatility of at the money option can be predicted first by looking at the implied volatilities over time (time-series analysis), and then by also including other features into the analysis. From a business perspective these predictions, if deemed successful, could be used to implement option trading strategies, to produce profits, based upon those predictions.

The purpose of the following notebook will be to clean and prepare data for an analysis of volatility in Twitter stock. The data on Twitter stock and options is available for free on quandl.com and is available to anyone who downloads an API key. The data on the stock itself is provided separately from the option data and includes items such as high, low, close, and adjusted (taking into account splits) stock price. The option data provides insight into both historical volatility of the stock as well as the implied volatility of options of different maturity lengths. In order to use this data it will have to be cleaned by eliminating entries with NaN's and organized into appropriate categories. Additionally, the stock and option data needs to be combined into a useful data frame.

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [47]:
# call quand API for INTC option volatilities

api = 'https://www.quandl.com/api/v3/datasets/EOD/INTC.csv?api_key=ydWv6UATm3p8xLJXKVCj'
INTC = pd.read_csv(api)
INTC['Date'] = pd.to_datetime(vol.Date)
INTC.head()

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
0,2017-12-28,46.36,46.36,45.95,46.22,9504372.0,0.0,1.0,42.812145,42.812145,42.433522,42.682859,9504372.0
1,2017-12-27,46.11,46.36,46.00,46.11,13359671.0,0.0,1.0,42.581277,42.812145,42.479695,42.581277,13359671.0
2,2017-12-26,46.28,46.47,45.95,46.08,15533328.0,0.0,1.0,42.738267,42.913727,42.433522,42.553573,15533328.0
3,2017-12-22,46.33,47.02,46.02,46.70,33913537.0,0.0,1.0,42.784441,43.421636,42.498165,43.126125,33913537.0
4,2017-12-21,47.54,47.59,46.56,46.76,42963451.0,0.0,1.0,43.901841,43.948015,42.996839,43.181534,42963451.0


In [48]:
INTC.shape

(1090, 13)

In [49]:
INTC.tail()

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
1085,2013-09-09,22.72,23.03,22.70,22.910,27770600.0,0.0,1.0,18.406685,18.657833,18.390482,18.560615,27770600.0
1086,2013-09-06,22.69,22.82,22.40,22.670,27606600.0,0.0,1.0,18.382381,18.487701,18.147436,18.366178,27606600.0
1087,2013-09-05,22.68,22.73,22.49,22.600,22552800.0,0.0,1.0,18.374279,18.414787,18.220350,18.309467,22552800.0
1088,2013-09-04,22.38,22.78,22.28,22.635,37941800.0,0.0,1.0,18.131233,18.455295,18.050218,18.337822,37941800.0
1089,2013-09-03,22.21,22.24,21.97,22.067,26427300.0,0.0,1.0,17.993507,18.017812,17.799070,17.877655,26427300.0


In [50]:
clean_INTC =INTC.dropna(axis = 'columns')
clean_INTC.head()

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
0,2017-12-28,46.36,46.36,45.95,46.22,9504372.0,0.0,1.0,42.812145,42.812145,42.433522,42.682859,9504372.0
1,2017-12-27,46.11,46.36,46.00,46.11,13359671.0,0.0,1.0,42.581277,42.812145,42.479695,42.581277,13359671.0
2,2017-12-26,46.28,46.47,45.95,46.08,15533328.0,0.0,1.0,42.738267,42.913727,42.433522,42.553573,15533328.0
3,2017-12-22,46.33,47.02,46.02,46.70,33913537.0,0.0,1.0,42.784441,43.421636,42.498165,43.126125,33913537.0
4,2017-12-21,47.54,47.59,46.56,46.76,42963451.0,0.0,1.0,43.901841,43.948015,42.996839,43.181534,42963451.0


In [51]:
INTC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1090 non-null   datetime64[ns]
 1   Open        1090 non-null   float64       
 2   High        1090 non-null   float64       
 3   Low         1090 non-null   float64       
 4   Close       1090 non-null   float64       
 5   Volume      1090 non-null   float64       
 6   Dividend    1090 non-null   float64       
 7   Split       1090 non-null   float64       
 8   Adj_Open    1090 non-null   float64       
 9   Adj_High    1090 non-null   float64       
 10  Adj_Low     1090 non-null   float64       
 11  Adj_Close   1090 non-null   float64       
 12  Adj_Volume  1090 non-null   float64       
dtypes: datetime64[ns](1), float64(12)
memory usage: 110.8 KB


In [52]:
clean_INTC = pd.DataFrame(INTC)
clean_INTC = clean_INTC.set_index('Date')
clean_INTC.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2017-12-28,46.36,46.36,45.95,46.22,9504372.0,0.0,1.0,42.812145,42.812145,42.433522,42.682859,9504372.0
2017-12-27,46.11,46.36,46.00,46.11,13359671.0,0.0,1.0,42.581277,42.812145,42.479695,42.581277,13359671.0
2017-12-26,46.28,46.47,45.95,46.08,15533328.0,0.0,1.0,42.738267,42.913727,42.433522,42.553573,15533328.0
2017-12-22,46.33,47.02,46.02,46.70,33913537.0,0.0,1.0,42.784441,43.421636,42.498165,43.126125,33913537.0
2017-12-21,47.54,47.59,46.56,46.76,42963451.0,0.0,1.0,43.901841,43.948015,42.996839,43.181534,42963451.0


In [53]:
prices = pd.DataFrame(INTC)
prices = prices[['Close', 'Date']]
prices = prices.set_index('Date')
prices.head(3)

,Close
Date,
2017-12-28,46.22
2017-12-27,46.11
2017-12-26,46.08
